In [ ]:
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null   
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz  
!tar xf spark-3.1.2-bin-hadoop3.2.tgz 
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

import findspark
findspark.init("spark-3.1.2-bin-hadoop3.2")
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql import Row
from pyspark.sql import functions

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,575 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Err:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY A4B469963BF863CC
Hit:10 http://ppa.launchpad.net/cran/lib

In [ ]:
spark_session = SparkSession.builder.appName("BigDataProject").getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Fact_ratings


In [ ]:
fact_ratings = spark_session.read.csv("/content/drive/MyDrive/TCBDA/Assignments/Big Data/BX-Book-Ratings.csv", sep='"', inferSchema=True, header=True, encoding="ISO-8859-1")

In [ ]:
print("Fact_ratings schema")
fact_ratings.printSchema()

print("Fact_ratings - first 20 rows")
fact_ratings.show()

Fact_ratings schema
root
 |-- _c0: string (nullable = true)
 |-- User-ID;: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- ;: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- Book-Rating: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- ,,: string (nullable = true)

Fact_ratings - first 20 rows
+----+--------+----+----------+----+---+----+-----------+----+----+---+
| _c0|User-ID;| _c2|      ISBN| _c4|  ;| _c6|Book-Rating| _c8| _c9| ,,|
+----+--------+----+----------+----+---+----+-----------+----+----+---+
|null| 276725;|null|034545104X|null|  ;|null|          0|null|null| ,,|
|null| 276726;|null|0155061224|null|  ;|null|          5|null|null| ,,|
|null| 276727;|null|0446520802|null|  ;|null|          0|null|null| ,,|
|null| 276729;|null|052165615X|null|  ;|null|          3|null|null| ,,|
|null| 276729;|null|0521795028|null|  ;|null|

In [ ]:
# Drop irrelevant columns from the book rating DataFrame
columns_to_drop = ["_c0", "_c2", "_c4", ";", "_c6", "_c8", "_c9", ",,"]
fact_ratings = fact_ratings.drop(*columns_to_drop)

# Rename the User-ID column in the Book Ratings DataFrame
fact_ratings = fact_ratings.withColumnRenamed("User-ID;", "UserBK")

# Rename the ISBN column in the Book Ratings DataFrame
fact_ratings = fact_ratings.withColumnRenamed("ISBN", "BookBK")

In [ ]:
print("Fact_ratings - description")
fact_ratings.describe().show()

Fact_ratings - description
+-------+-------+-----------+------------------+
|summary| UserBK|     BookBK|       Book-Rating|
+-------+-------+-----------+------------------+
|  count|1048575|    1048574|           1048567|
|   mean|   null|   Infinity|2.8798762111848633|
| stddev|   null|        NaN|3.8578638469186437|
|    min|100001;| 0330299891|                 0|
|    max|     9;|  Ô½crosoft|                 ;|
+-------+-------+-----------+------------------+



In [ ]:
# Drop null values from the book ratings DataFrame
fact_ratings.filter("UserBK IS NULL OR BookBK IS NULL OR 'Book-Rating' IS NULL").show()
fact_ratings = fact_ratings.na.drop()
fact_ratings.filter("UserBK IS NULL OR BookBK IS NULL OR 'Book-Rating' IS NULL").show()

+-------+------+-----------+
| UserBK|BookBK|Book-Rating|
+-------+------+-----------+
|130499;|  null|          ;|
+-------+------+-----------+

+------+------+-----------+
|UserBK|BookBK|Book-Rating|
+------+------+-----------+
+------+------+-----------+



In [ ]:
# Remove the ";" from the User-BK column by using translate()
fact_ratings = fact_ratings.withColumn("UserBK", functions.translate(fact_ratings["UserBK"], ";", ""))

# Change the column data types of the "User-BK" and "Book-Rating" columns to "int"
fact_ratings = fact_ratings.withColumn("UserBK",fact_ratings["UserBK"].cast("int"))
fact_ratings = fact_ratings.withColumn("Book-Rating",fact_ratings["Book-Rating"].cast("int"))
fact_ratings.printSchema()

# We can see now that the "User-ID" has mean and stddev values, unlike before also that the ";" values have been dropped from the "Book-Rating" column
fact_ratings.describe().show()

root
 |-- UserBK: integer (nullable = true)
 |-- BookBK: string (nullable = true)
 |-- Book-Rating: integer (nullable = true)

+-------+------------------+-----------+------------------+
|summary|            UserBK|     BookBK|       Book-Rating|
+-------+------------------+-----------+------------------+
|  count|           1048566|    1048566|           1048560|
|   mean|128509.03468928041|   Infinity|2.8798762111848633|
| stddev| 74218.66800140661|        NaN|3.8578638469186437|
|    min|                 2| 0330299891|                 0|
|    max|            278854|  Ô½crosoft|                10|
+-------+------------------+-----------+------------------+



In [ ]:
# Remove rows where BookBK length isn't 10 (the standard length)
print(fact_ratings.where(functions.length(functions.col("BookBK")) != 10).count())
fact_ratings = fact_ratings.where(functions.col("BookBK") != 10)

9429


In [ ]:
# Remove all rows where BookBK contains spaces or dots
print(fact_ratings.filter(functions.col("BookBK").contains(" ")).count())
fact_ratings = fact_ratings.where(~functions.col("BookBK").contains(" "))

print(fact_ratings.filter(functions.col("BookBK").contains(".")).count())
fact_ratings = fact_ratings.where(~functions.col("BookBK").contains("."))

4
320


In [ ]:
# Add a "RatingSK" surrogate key in Fact_ratings
fact_ratings = fact_ratings.withColumn("RatingSK", functions.monotonically_increasing_id() + 100)
fact_ratings.show()

+------+----------+-----------+--------+
|UserBK|    BookBK|Book-Rating|RatingSK|
+------+----------+-----------+--------+
|276726|0155061224|          5|     100|
|276727|0446520802|          0|     101|
|276729|0521795028|          6|     102|
|276733|2080674722|          0|     103|
|276737|0600570967|          6|     104|
|276745| 342310538|         10|     105|
|276746|0425115801|          0|     106|
|276746|0449006522|          0|     107|
|276746|0553561618|          0|     108|
|276746|0786013990|          0|     109|
|276746|0786014512|          0|     110|
|276747|0060517794|          9|     111|
|276747|0451192001|          0|     112|
|276747|0609801279|          0|     113|
|276747|0671537458|          9|     114|
|276747|0679776818|          8|     115|
|276747|0943066433|          7|     116|
|276747|1570231028|          0|     117|
|276747|1885408226|          7|     118|
|276748|0747558167|          6|     119|
+------+----------+-----------+--------+
only showing top

In [ ]:
fact_ratings.coalesce(1).write.option("header", True).csv("/content/drive/MyDrive/TCBDA/Assignments/Big Data/fact_ratings")

#Dim_books

In [ ]:
dim_books = spark_session.read.csv("/content/drive/MyDrive/TCBDA/Assignments/Big Data/BX-Books.csv", sep=";", inferSchema=True, header=True, encoding="ISO-8859-1")

In [ ]:
print("Dim_books schema")
dim_books.printSchema()

print("Dim_books - first 20 rows")
dim_books.show()

Dim_books schema
root
 |-- ISBN: string (nullable = true)
 |-- Book-Title: string (nullable = true)
 |-- Book-Author: string (nullable = true)
 |-- Year-Of-Publication: integer (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Image-URL-S: string (nullable = true)
 |-- Image-URL-M: string (nullable = true)
 |-- Image-URL-L: string (nullable = true)

Dim_books - first 20 rows
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|      ISBN|          Book-Title|         Book-Author|Year-Of-Publication|           Publisher|         Image-URL-S|         Image-URL-M|         Image-URL-L|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|               2002|Oxford University...|http://images.ama...|http://images.a

In [ ]:
# Drop irrelevant columns from the books DataFrame
columns_to_drop = ["Image-URL-S", "Image-URL-M", "Image-URL-L"]
dim_books = dim_books.drop(*columns_to_drop)

# Rename the ISBN column in the Books DataFrame
dim_books = dim_books.withColumnRenamed("ISBN", "BookBK")

In [ ]:
print("Dim_books - description")
dim_books.describe().show()

Dim_books - description
+-------+--------------------+--------------------+--------------+-------------------+--------------------+
|summary|              BookBK|          Book-Title|   Book-Author|Year-Of-Publication|           Publisher|
+-------+--------------------+--------------------+--------------+-------------------+--------------------+
|  count|              271379|              271379|        271379|             271379|              271379|
|   mean|1.0412353852604752E9|            Infinity|          null| 1959.7560496574902|  25032.333333333332|
| stddev|1.4877523499740484E9|                 NaN|          null|  258.0113625638109|   21676.03850645531|
|    min|          0000913154| A Light in the S...|      D. Chiel|                  0| Editions P. Terrail|
|    max|          B0002K6K8O|   Ã?Â?thique en toc|Ã?Â?ric Holder|               2050|      Ã?Â¶bv&amp;hpt|
+-------+--------------------+--------------------+--------------+-------------------+--------------------+



In [ ]:
# The dim_books DataFrame has no null values
dim_books.filter("BookBK IS NULL OR 'Book-Title' IS NULL OR 'Book-Author' IS NULL OR 'Year-Of-Publication' IS NULL OR Publisher IS NULL").show()

+------+----------+-----------+-------------------+---------+
|BookBK|Book-Title|Book-Author|Year-Of-Publication|Publisher|
+------+----------+-----------+-------------------+---------+
+------+----------+-----------+-------------------+---------+



In [ ]:
# Find future years in the Year-Of-Publication column and remove them
print(dim_books.filter(dim_books["Year-Of-Publication"] >= 2005).count())
dim_books = dim_books.where(functions.col("Year-Of-Publication") <= 2004)

72


In [ ]:
# There are a lot of rows where the "Year-Of-Publication" is 0, we should remove them
dim_books.filter(dim_books["Year-Of-Publication"] == 0).show()
print(dim_books.filter(dim_books["Year-Of-Publication"] == 0).count())
dim_books = dim_books.where(functions.col("Year-Of-Publication") != 0)

+----------+--------------------+--------------------+-------------------+--------------------+
|    BookBK|          Book-Title|         Book-Author|Year-Of-Publication|           Publisher|
+----------+--------------------+--------------------+-------------------+--------------------+
|3150000335|    Kabale Und Liebe|            Schiller|                  0|Philipp Reclam, J...|
|342311360X|Die Liebe in Den ...|Gabriel Garcia Ma...|                  0|Deutscher Taschen...|
|0571197639|Poisonwood Bible ...|  Barbara Kingsolver|                  0|     Faber Faber Inc|
|3596214629|Herr Der Fliegen ...|             Golding|                  0|Fischer Taschenbu...|
|8845229041|Biblioteca Univer...|            P Coelho|                  0|  Fabbri - RCS Libri|
|0345251733|Rainbow the Storm...|   Christopher Finch|                  0|    Ballantine Books|
|0590021117|Little House On t...| Laura Ingall Wilder|                  0|      Scholastic Inc|
|2070362388|              Ravage|       

In [ ]:
# Keep only rows where "BookBK" is of length 10
print(dim_books.where(functions.length(functions.col("BookBK")) != 10).count())
dim_books = dim_books.where(functions.col("BookBK") != 10)

4


In [ ]:
# Drop duplicates from the BookBK column 
dim_books = dim_books.dropDuplicates(["BookBK"])

In [ ]:
# Remove some rows where "Book-Author" includes the substring '\"'
print(dim_books.filter(functions.col("Book-Author").contains('\"')).count())
dim_books = dim_books.where(~functions.col("Book-Author").contains('\"'))

43


In [ ]:
# Add a "BookSK" surrogate key in Dim_books
dim_books = dim_books.withColumn("BookSK", functions.monotonically_increasing_id() + 100)
dim_books.show()

+----------+--------------------+--------------------+-------------------+--------------------+------+
|    BookBK|          Book-Title|         Book-Author|Year-Of-Publication|           Publisher|BookSK|
+----------+--------------------+--------------------+-------------------+--------------------+------+
|0002234947|Miss Hobbema Pageant|       W.P. Kinsella|               1990|Harpercollins Juv...|   100|
|0002243687|Brother Frank's g...|       W. P Kinsella|               1994|       HarperCollins|   101|
|0002257084|          Dolce Vita|        Iseult Teran|               1999|HarperCollins Pub...|   102|
|0007640617|  Energize Your Life|          Nic Rowley|               2002| Thorsons Publishers|   103|
|0020444400|The voyage of the...|          C. S Lewis|               1986|             Collier|   104|
|0025483307|Stranger in Two W...|         Jean Harris|               1986|Simon &amp; Schuster|   105|
|0028610881|The Complete Idio...|      Mary Ann Young|               1997

In [ ]:
dim_books.coalesce(1).write.option("header", True).csv("/content/drive/MyDrive/TCBDA/Assignments/Big Data/dim_books")

#Dim_users

In [ ]:
dim_users = spark_session.read.csv("/content/drive/MyDrive/TCBDA/Assignments/Big Data/BX-Users.csv", sep=";", inferSchema=True, header=True, encoding="ISO-8859-1")

In [ ]:
print("Dim_users schema")
dim_users.printSchema()

print("Dim_users - first 20 rows")
dim_users.show()

Dim_users schema
root
 |-- User-ID: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: string (nullable = true)

Dim_users - first 20 rows
+-------+--------------------+----+
|User-ID|            Location| Age|
+-------+--------------------+----+
|      1|  nyc, new york, usa|NULL|
|      2|stockton, califor...|  18|
|      3|moscow, yukon ter...|NULL|
|      4|porto, v.n.gaia, ...|  17|
|      5|farnborough, hant...|NULL|
|      6|santa monica, cal...|  61|
|      7| washington, dc, usa|NULL|
|      8|timmins, ontario,...|NULL|
|      9|germantown, tenne...|NULL|
|     10|albacete, wiscons...|  26|
|     11|melbourne, victor...|  14|
|     12|fort bragg, calif...|NULL|
|     13|barcelona, barcel...|  26|
|     14|mediapolis, iowa,...|NULL|
|     15|calgary, alberta,...|NULL|
|     16|albuquerque, new ...|NULL|
|     17|chesapeake, virgi...|NULL|
|     18|rio de janeiro, r...|  25|
|     19|           weston, ,|  14|
|     20|langhorne, pennsy...|  19|
+-------+

In [ ]:
# Rename the User-ID column in the dim_users DataFrame
dim_users = dim_users.withColumnRenamed("User-ID","UserBK")

In [ ]:
print("Dim_users - description")
dim_users.describe().show()

Dim_users - description
+-------+-----------------+--------------------+-----------------+
|summary|           UserBK|            Location|              Age|
+-------+-----------------+--------------------+-----------------+
|  count|           278859|              278859|           278857|
|   mean|         139429.5|                null|34.75143370454978|
| stddev|80499.51502027822|                null|14.42809738245543|
|    min|  , milan, italy"|"alexandria"., "a...|                0|
|    max|            99999|  ýzmýr, n/a, turkey|             NULL|
+-------+-----------------+--------------------+-----------------+



In [ ]:
# Drop null values from the dim_users DataFrame
dim_users.filter("UserBK IS NULL OR Location IS NULL OR Age IS NULL").show()
dim_users = dim_users.na.drop()
dim_users.filter("UserBK IS NULL OR Location IS NULL OR Age IS NULL").show()

+---------------+----------+----+
|         UserBK|  Location| Age|
+---------------+----------+----+
|         275081|cernusco s|null|
|, milan, italy"|      NULL|null|
+---------------+----------+----+

+------+--------+---+
|UserBK|Location|Age|
+------+--------+---+
+------+--------+---+



In [ ]:
# Change the column data types of the "UserBK" and "Age" columns to "int"
dim_users = dim_users.withColumn("UserBK", dim_users["UserBK"].cast("int"))
age_as_int = dim_users.withColumn("Age", dim_users["Age"].cast("int"))

# Replace all null values in the "Age" column with -1
age_as_int.count()
dim_users = age_as_int.fillna(-1)

dim_users.printSchema()
dim_users.show()

root
 |-- UserBK: integer (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: integer (nullable = true)

+------+--------------------+---+
|UserBK|            Location|Age|
+------+--------------------+---+
|     1|  nyc, new york, usa| -1|
|     2|stockton, califor...| 18|
|     3|moscow, yukon ter...| -1|
|     4|porto, v.n.gaia, ...| 17|
|     5|farnborough, hant...| -1|
|     6|santa monica, cal...| 61|
|     7| washington, dc, usa| -1|
|     8|timmins, ontario,...| -1|
|     9|germantown, tenne...| -1|
|    10|albacete, wiscons...| 26|
|    11|melbourne, victor...| 14|
|    12|fort bragg, calif...| -1|
|    13|barcelona, barcel...| 26|
|    14|mediapolis, iowa,...| -1|
|    15|calgary, alberta,...| -1|
|    16|albuquerque, new ...| -1|
|    17|chesapeake, virgi...| -1|
|    18|rio de janeiro, r...| 25|
|    19|           weston, ,| 14|
|    20|langhorne, pennsy...| 19|
+------+--------------------+---+
only showing top 20 rows



In [ ]:
# Find and drop all rows where the "Age" column is not between 10 and 100 (except -1)
print(dim_users.filter("(Age < 10 OR Age > 100) AND Age != -1").count())
dim_users = dim_users.filter((dim_users.Age  == -1) | ((dim_users.Age  >= 10) & (dim_users.Age  <= 100)))

1435


In [ ]:
# Create a "City" column
dim_users = dim_users.withColumn("City", functions.substring_index(dim_users["Location"], ", ", 1))


# Create a "Region" column
dim_users = dim_users.withColumn("Temp", functions.substring_index(dim_users["Location"], ", ", -2))
dim_users = dim_users.withColumn("Region", functions.substring_index(dim_users["Temp"], ", ", 1))


# Create a "Country" column
dim_users = dim_users.withColumn("Country", functions.substring_index(dim_users["Location"], ", ", -1))

# Drop irrelevant columns from the dim_users DataFrame
columns_to_drop = ["Location", "Temp"]
dim_users = dim_users.drop(*columns_to_drop)

dim_users.show()

+------+---+--------------+---------------+--------------+
|UserBK|Age|          City|         Region|       Country|
+------+---+--------------+---------------+--------------+
|     1| -1|           nyc|       new york|           usa|
|     2| 18|      stockton|     california|           usa|
|     3| -1|        moscow|yukon territory|        russia|
|     4| 17|         porto|       v.n.gaia|      portugal|
|     5| -1|   farnborough|          hants|united kingdom|
|     6| 61|  santa monica|     california|           usa|
|     7| -1|    washington|             dc|           usa|
|     8| -1|       timmins|        ontario|        canada|
|     9| -1|    germantown|      tennessee|           usa|
|    10| 26|      albacete|      wisconsin|         spain|
|    11| 14|     melbourne|       victoria|     australia|
|    12| -1|    fort bragg|     california|           usa|
|    13| 26|     barcelona|      barcelona|         spain|
|    14| -1|    mediapolis|           iowa|           us

In [ ]:
# Replace all values of "usa" with "united states" in the Country column, because later on this country appears as "united states" in the countries DataFrame
dim_users = dim_users.withColumn("Country", functions.regexp_replace("Country", "usa", "united states"))

In [ ]:
# Find out how many empty values there are in these columns
print(dim_users.filter("City = ''").count())
print(dim_users.filter("Region = ''").count())
print(dim_users.filter("Country = ''").count())

# Drop all of these rows
dim_users = dim_users.where(functions.col("City") != "")
dim_users = dim_users.where(functions.col("Region") != "")
dim_users = dim_users.where(functions.col("Country") != "")

120
803
0


In [ ]:
# Drop all numeric values from the "City" column
print(dim_users.filter(functions.col("City").rlike("^[0-9]*$")).count())
dim_users = dim_users.where(~functions.col("City").rlike("^[0-9]*$"))

# Drop all rows where the "City" column is "n/a"
print(dim_users.filter("City = 'n/a'").count())
dim_users = dim_users.where(functions.col("City") != "n/a")

# Drop all rows where the "Region" column is "n/a"
print(dim_users.filter("Region = 'n/a'").count())
dim_users = dim_users.where(functions.col("Region") != "n/a")

56
430
12112


In [ ]:
# Drop all rows that start or end with a special character in the "City", "Region" and "Country" columns
characters = ["-", "?", ".", '"', "(", "*", "&", "<", ">", "~", "|", "/", "_", "#", "!", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
for character in characters:
  dim_users = dim_users.where(~functions.col("City").startswith(character))
  dim_users = dim_users.where(~functions.col("Region").startswith(character))
  dim_users = dim_users.where(~functions.col("Country").startswith(character))

In [ ]:
pip install pycountry

     |████████████████████████████████| 10.1 MB 14.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=24071202a05c5dd33531c465af9c7b2d07d98ba22b95fbea94a92e1ba1e00f2b
  Stored in directory: /root/.cache/pip/wheels/0e/06/e8/7ee176e95ea9a8a8c3b3afcb1869f20adbd42413d4611c6eb4
Successfully built pycountry


In [ ]:
# Create a countries DataFrame

from pycountry import countries

countries_list = list(countries)
countries_names = []
for country in countries_list:
  countries_names.append([country.name.lower()])

columns = ["Country2"]
countries_df = spark_session.createDataFrame(countries_names, columns)
countries_df.show()
countries_df.printSchema()


+--------------------+
|            Country2|
+--------------------+
|               aruba|
|         afghanistan|
|              angola|
|            anguilla|
|       åland islands|
|             albania|
|             andorra|
|united arab emirates|
|           argentina|
|             armenia|
|      american samoa|
|          antarctica|
|french southern t...|
| antigua and barbuda|
|           australia|
|             austria|
|          azerbaijan|
|             burundi|
|             belgium|
|               benin|
+--------------------+
only showing top 20 rows

root
 |-- Country2: string (nullable = true)



In [ ]:
# Create an inner join betweem dim_users and the countries DataFrame in order to drop all rows where the country value is illegal
dim_users = dim_users.join(countries_df, dim_users["Country"] == countries_df["Country2"], "inner")
columns_to_drop = ["Country2"]
dim_users = dim_users.drop(*columns_to_drop)
dim_users.show()

+------+---+-------------+------------------+--------+
|UserBK|Age|         City|            Region| Country|
+------+---+-------------+------------------+--------+
|247783| 14|           na|                na|   aruba|
|273722| -1|     sta cruz|             aruba|   aruba|
|  9304| 47|    port soif|              vale|guernsey|
| 71729| -1|   albersdorf|schleswig-holstein|guernsey|
| 76791| 27|st peter port|   channel islands|guernsey|
|152577| 27|st peter port|   channel islands|guernsey|
|159093| 51|wilhelmshaven|     niedersachsen|guernsey|
|188519| -1|      cddsdsd|           abruzzo|guernsey|
|198595| -1|  saarbrücken|          saarland|guernsey|
|205572| -1|     guernsey|   channel islands|guernsey|
|270309| 23|       dessau|            berlin|guernsey|
|    96| -1|     helsinki|            alaska| finland|
|   233| -1|      tampere|         pirkanmaa| finland|
|   307| 27|        koski|           uusimaa| finland|
|   339| 20|      tampere|         pirkanmaa| finland|
|  2196| 2

In [ ]:
# Add a "UserSK" surrogate key in Dim_users
dim_users = dim_users.withColumn("UserSK", functions.monotonically_increasing_id() + 100)
dim_users.show()

+------+---+-------------+------------------+--------+----------+
|UserBK|Age|         City|            Region| Country|    UserSK|
+------+---+-------------+------------------+--------+----------+
|247783| 14|           na|                na|   aruba|       100|
|273722| -1|     sta cruz|             aruba|   aruba|       101|
|  9304| 47|    port soif|              vale|guernsey|       102|
| 71729| -1|   albersdorf|schleswig-holstein|guernsey|       103|
| 76791| 27|st peter port|   channel islands|guernsey|       104|
|152577| 27|st peter port|   channel islands|guernsey|       105|
|159093| 51|wilhelmshaven|     niedersachsen|guernsey|       106|
|188519| -1|      cddsdsd|           abruzzo|guernsey|       107|
|198595| -1|  saarbrücken|          saarland|guernsey|       108|
|205572| -1|     guernsey|   channel islands|guernsey|       109|
|270309| 23|       dessau|            berlin|guernsey|       110|
|    96| -1|     helsinki|            alaska| finland|8589934692|
|   233| -

In [ ]:
dim_users.coalesce(1).write.option("header", True).csv("/content/drive/MyDrive/TCBDA/Assignments/Big Data/dim_users")